In [380]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_columns', None)  # This will display all columns


In [381]:
dfc = pd.read_csv('calendar.csv')
dfl = pd.read_csv('listings.csv')
dfr = pd.read_csv('reviews.csv')

# Airbnb Listings Data in Seattle

The following datasets focus on Airbnb listings data in Seattle:

1. **`dfc`**: Calendars Data
   - Lists each listing's daily availability and price between January 2016 and January 2017.


2. **`dfl`**: Detailed Listing Information
   - Includes data on prices, fees, location, amenities, and reviews for each listing.


3. **`dfr`**: Reviews Data
   - Contains detailed information about the reviews given to the listings, including ratings and text reviews.


# Target Research Questions

The following are the three target research questions:

1. What months witnessed the peak of the average listing nightly price in Seattle?
2. What are the top 3 listing neighborhoods in terms of average nightly price in Seattle?
3. What factors are most correlated with predicting the listing nightly price?


# Analysis Plan

- To answer Q1, we'll use the `dfc` dataset.
- To answer Q2 and Q3, we'll use the `dfl` dataset.
- The `dfr` dataset will not be necessary for this analysis, as we have ratings score data per listing in `dfl`; therefore, it will not be needed.


# Step 1: Data Assessment

## 1. dfc

In [382]:
print(dfc.head())
print(dfc.shape)

   listing_id        date available   price
0      241032  2016-01-04         t  $85.00
1      241032  2016-01-05         t  $85.00
2      241032  2016-01-06         f     NaN
3      241032  2016-01-07         f     NaN
4      241032  2016-01-08         f     NaN
(1393570, 4)


In [384]:
#checking for data types
dfc.dtypes

listing_id     int64
date          object
available     object
price         object
dtype: object

In [385]:
dfc.describe()

,listing_id
count,1.393570e+06
mean,5.550111e+06
std,2.962274e+06
min,3.335000e+03
25%,3.258213e+06
50%,6.118244e+06
75%,8.035212e+06
max,1.034016e+07


In [386]:
#checking for nulls
dfc.isna().sum()

listing_id         0
date               0
available          0
price         459028
dtype: int64

In [387]:
#comparing to see if the nulls in price is only when it's not available
dfc[dfc['available']=='f'].isna().sum()

listing_id         0
date               0
available          0
price         459028
dtype: int64

In [406]:
#checking for duplicates
dfc.duplicated().sum()

0

### dfc Quality Assessment

- The `dfc` dataset is very clean; however, the data types need to be fixed.
- The null values in the price column are logical, as they indicate that the listing is not available for booking on the given day.
- During data wrangling, these rows will be removed.
- No duplicate rows found.


## 2. dfl

In [393]:
print(dfl.head())
print(dfl.shape)

        id                           listing_url       scrape_id last_scraped  \
0   241032   https://www.airbnb.com/rooms/241032  20160104002432   2016-01-04   
1   953595   https://www.airbnb.com/rooms/953595  20160104002432   2016-01-04   
2  3308979  https://www.airbnb.com/rooms/3308979  20160104002432   2016-01-04   
3  7421966  https://www.airbnb.com/rooms/7421966  20160104002432   2016-01-04   
4   278830   https://www.airbnb.com/rooms/278830  20160104002432   2016-01-04   

                                  name  \
0         Stylish Queen Anne Apartment   
1   Bright & Airy Queen Anne Apartment   
2  New Modern House-Amazing water view   
3                   Queen Anne Chateau   
4       Charming craftsman 3 bdm house   

                                             summary  \
0                                                NaN   
1  Chemically sensitive? We've removed the irrita...   
2  New modern house built in 2013.  Spectacular s...   
3  A charming apartment that sits at

In [394]:
#checking for data types
dfl.dtypes

id                                    int64
listing_url                          object
scrape_id                             int64
last_scraped                         object
name                                 object
                                     ...   
cancellation_policy                  object
require_guest_profile_picture        object
require_guest_phone_verification     object
calculated_host_listings_count        int64
reviews_per_month                   float64
Length: 92, dtype: object

In [395]:
dfl.describe()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,square_feet,guests_included,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,reviews_per_month
count,3.818000e+03,3.818000e+03,3.818000e+03,3816.000000,3816.000000,3818.000000,3818.000000,3818.000000,3802.000000,3812.000000,3817.000000,97.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3818.000000,3171.000000,3160.000000,3165.000000,3160.000000,3167.000000,3163.000000,3162.000000,0.0,3818.000000,3191.000000
mean,5.550111e+06,2.016010e+13,1.578556e+07,7.157757,7.157757,47.628961,-122.333103,3.349398,1.259469,1.307712,1.735394,854.618557,1.672603,2.369303,780.447617,16.786276,36.814825,58.082504,244.772656,22.223415,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,NaN,2.946307,2.078919
std,2.962660e+06,0.000000e+00,1.458382e+07,28.628149,28.628149,0.043052,0.031745,1.977599,0.590369,0.883395,1.139480,671.404893,1.311040,16.305902,1683.589007,12.173637,23.337541,34.063845,126.772526,37.730892,6.606083,0.698031,0.797274,0.595499,0.568211,0.629053,0.750259,NaN,5.893029,1.822348
min,3.335000e+03,2.016010e+13,4.193000e+03,1.000000,1.000000,47.505088,-122.417219,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,2.000000,3.000000,2.000000,2.000000,4.000000,2.000000,NaN,1.000000,0.020000
25%,3.258256e+06,2.016010e+13,3.275204e+06,1.000000,1.000000,47.609418,-122.354320,2.000000,1.000000,1.000000,1.000000,420.000000,1.000000,1.000000,60.000000,2.000000,13.000000,28.000000,124.000000,2.000000,93.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,NaN,1.000000,0.695000
50%,6.118244e+06,2.016010e+13,1.055814e+07,1.000000,1.000000,47.623601,-122.328874,3.000000,1.000000,1.000000,1.000000,750.000000,1.000000,2.000000,1125.000000,20.000000,46.000000,73.000000,308.000000,9.000000,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,1.000000,1.540000
75%,8.035127e+06,2.016010e+13,2.590309e+07,3.000000,3.000000,47.662694,-122.310800,4.000000,1.000000,2.000000,2.000000,1200.000000,2.000000,2.000000,1125.000000,30.000000,59.000000,89.000000,360.000000,26.000000,99.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,2.000000,3.000000
max,1.034016e+07,2.016010e+13,5.320861e+07,502.000000,502.000000,47.733358,-122.240607,16.000000,8.000000,7.000000,15.000000,3000.000000,15.000000,1000.000000,100000.000000,30.000000,60.000000,90.000000,365.000000,474.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,37.000000,12.150000


In [407]:
# Too many columns to display, viewing them alone
dfl.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [397]:
#checking for nulls
dfl.isna().sum()

id                                    0
listing_url                           0
scrape_id                             0
last_scraped                          0
name                                  0
                                   ... 
cancellation_policy                   0
require_guest_profile_picture         0
require_guest_phone_verification      0
calculated_host_listings_count        0
reviews_per_month                   627
Length: 92, dtype: int64

In [408]:
#creating a separate df to view columns with nulls
na_summary = dfl.isna().sum().reset_index()
na_summary.columns = ['Column', 'Missing Values']
na_summary = na_summary[na_summary['Missing Values'] != 0]
na_summary

,Column,Missing Values
5,summary,177
6,space,569
9,neighborhood_overview,1032
10,notes,1606
11,transit,934
12,thumbnail_url,320
13,medium_url,320
15,xl_picture_url,320
18,host_name,2
19,host_since,2


In [413]:
#creating a separate df to view columns dtypes
pd.set_option('display.max_rows', None)

dtype_summary = dfl.dtypes.reset_index()
dtype_summary.columns = ['Column', 'dtype']
dtype_summary

,Column,dtype
0,id,int64
1,listing_url,object
2,scrape_id,int64
3,last_scraped,object
4,name,object
5,summary,object
6,space,object
7,description,object
8,experiences_offered,object
9,neighborhood_overview,object


In [404]:
dfl.duplicated().sum()

0

# dfl Quality Assessment

- The `dfl` dataset has 92 columns, a lot of the columns are duplicates of eachother, and many of them will not provide value to the analysis, therefore they'll be dropped.
- Null values are present in a lot of columns, they'll be dealt with after dropping the unnecessary features first.
- dtypes will need some augmenting, especially for numerical features that are being stores as objects, and some times as floats instead of integers.
- No duplicates.
